# Symptom Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import zeep
from zeep import Client
from lxml import etree
from zeep import Plugin

In [2]:
#Read in data
df = pd.read_csv('symptoms.csv', sep=',',encoding='latin-1')

In [3]:
df.reset_index(drop=True)

,Unnamed: 0,[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].[MEMBER_CAPTION],[PathogenMitKrankheit].[Meldeweg Web71].[Krankheit ID Web71].[MEMBER_CAPTION],[Symptome].[ID].[ID].[MEMBER_CAPTION],[ReportingDate].[YearWeek].[YearWeek].[MEMBER_CAPTION],[Measures].[FallCount]
0,1,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW13,1
1,2,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW33,1
2,3,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW44,1
3,4,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW53,1
4,5,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW05,1
5,6,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW06,5
6,7,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW07,3
7,8,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW08,4
8,9,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW10,7
9,10,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW11,5


In [4]:
#Drop unecessary columns
df = df.drop(columns=['Unnamed: 0','[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].[MEMBER_CAPTION]'])

In [5]:
#Give columns more comprehensible names
df = df.rename(columns={'[PathogenMitKrankheit].[Meldeweg Web71].[Krankheit ID Web71].[MEMBER_CAPTION]': 'Krankheit', '[Symptome].[ID].[ID].[MEMBER_CAPTION]': 'Symptom', '[ReportingDate].[YearWeek].[YearWeek].[MEMBER_CAPTION]': 'Kalenderwoche', '[Measures].[FallCount]': 'Anzahl'}) 


In [6]:
#Since we want to focus only on Norovirus, Infulenza, and Windpocken, we throw all the other data away
df_selection = df[df.isin({'Krankheit': ['Norovirus-Gastroenteritis','Influenza', 'Windpocken']})['Krankheit']]

In [7]:
#df_selection_symptoms = df_selection.groupby(['Krankheit','Symptom'])['Anzahl'].sum().reset_index()
#df_selection_symptoms

In [8]:
#Further dropping
df_selection = df_selection.drop(columns='Krankheit')

In [9]:
df_selection_pivot = df_selection.groupby(['Symptom','Kalenderwoche'])['Anzahl'].sum().reset_index()
df_selection_pivot.sort_values('Kalenderwoche')

,Symptom,Kalenderwoche,Anzahl
3118,Husten,2001-KW01,2
6105,andere Symptome,2001-KW01,1
2055,Fieber,2001-KW01,3
3904,"Muskel-, Glieder-, Rücken- oder Kopfschmerzen",2001-KW01,1
253,Ausschlag an Haut oder Schleimhaut mit gleichz...,2001-KW01,10
3905,"Muskel-, Glieder-, Rücken- oder Kopfschmerzen",2001-KW02,5
3119,Husten,2001-KW02,3
254,Ausschlag an Haut oder Schleimhaut mit gleichz...,2001-KW02,15
2056,Fieber,2001-KW02,6
3120,Husten,2001-KW03,7


In [10]:
#get a list of all relevant symptoms
df_selection_all_symptoms = df_selection.groupby(['Symptom'])['Anzahl'].sum().reset_index()['Symptom'].tolist()

In [11]:
#for each combination of symptom and week:
#if the combination is not in the dataframe, then insert a row with this combination and 'Anzahl' = 0
new_rows = []
for week in df['Kalenderwoche'].drop_duplicates().sort_values():
    for symptom in df_selection_all_symptoms:
        sample_week = df_selection_pivot[df_selection_pivot['Kalenderwoche'] == week]
        if not sample_week.isin({'Symptom': [symptom]})['Symptom'].any():
            new_row = [symptom, week, 0]
            new_rows.append(new_row)
            
            
new_values = pd.DataFrame(new_rows, columns=['Symptom','Kalenderwoche','Anzahl'])

#append these illnesses with 0 'Anzahl'
df_selection_pivot= df_selection_pivot.append(new_values)
df_weekly_symptoms = df_selection_pivot.sort_values('Kalenderwoche').reset_index(drop=True)
df_weekly_symptoms = df_weekly_symptoms.sort_values(['Kalenderwoche','Symptom'],ascending=[True,True])

In [16]:
# Reset the index for ease of further processing
df_weekly_symptoms = df_weekly_symptoms.reset_index(drop=True)
print(df_weekly_symptoms.head(20))

                                              Symptom Kalenderwoche  Anzahl
0   Ausschlag an Haut oder Schleimhaut mit Flecken...     2001-KW01       0
1   Ausschlag an Haut oder Schleimhaut mit gleichz...     2001-KW01      10
2   Ausschlag, einseitig auf Hautsegment beschränk...     2001-KW01       0
3                                   Brennen, Juckreiz     2001-KW01       0
4                        Durchfall und/oder Erbrechen     2001-KW01       0
5                   Durchfall, nicht näher bezeichnet     2001-KW01       0
6            Fetales (kongenitales) Varizellensyndrom     2001-KW01       0
7                                              Fieber     2001-KW01       3
8                                    Gliederschmerzen     2001-KW01       0
9                                              Husten     2001-KW01       2
10      Muskel-, Glieder-, Rücken- oder Kopfschmerzen     2001-KW01       1
11                       Pneumonie (Lungenentzündung)     2001-KW01       0
12  Schmerze

In [17]:
df_weekly_symptoms.to_pickle('weekly_symptoms.pkl')